# Exploration des données

Importation des bibliothèques

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import *

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

Importation des données

In [6]:
df=pd.read_csv("mars-2014-complete.csv",encoding='ISO-8859-1', sep = ';')

Dimension des données

In [7]:
# Dimensions du DataFrame
print("Dimensions (lignes, colonnes) :", df.shape)

Dimensions (lignes, colonnes) : (55044, 30)


Infos sur le dataframe

In [17]:
display(df.head())
display(df.info())

# Nous avons 5 variables de type float64, 3 de type int64 et 22 de type object

,lib_mrq,lib_mod_doss,lib_mod,dscom,cnit,tvv,cod_cbr,hybride,puiss_admin_98,puiss_max,...,masse_ordma_min,masse_ordma_max,champ_v9,date_maj,Carrosserie,gamme,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,ALFA-ROMEO,159,159,159 1750 Tbi (200ch),M10ALFVP000G340,939AXN1B52C,ES,non,12,147,...,1505,1505,715/2007*692/2008EURO5,mars-14,BERLINE,MOY-SUPER,NaN,NaN,NaN,NaN
1,ALFA-ROMEO,159,159,159 1750 Tbi (200ch),M10ALFVP000H341,939BXN1B53C,ES,non,12,147,...,1555,1555,715/2007*692/2008EURO5,mars-14,BERLINE,MOY-SUPER,NaN,NaN,NaN,NaN
2,ALFA-ROMEO,159,159,159 2.0 JTDm (136ch),M10ALFVP000E302,939AXR1B64,GO,non,7,100,...,1565,1565,715/2007*692/2008EURO5,mars-14,BERLINE,MOY-SUPER,NaN,NaN,NaN,NaN
3,ALFA-ROMEO,159,159,159 2.0 JTDm (136ch),M10ALFVP000F303,939AXR1B64B,GO,non,7,100,...,1565,1565,715/2007*692/2008EURO5,mars-14,BERLINE,MOY-SUPER,NaN,NaN,NaN,NaN
4,ALFA-ROMEO,159,159,159 2.0 JTDm (170ch),M10ALFVP000G304,939AXS1B66,GO,non,9,125,...,1565,1565,715/2007*692/2008EURO5,mars-14,BERLINE,MOY-SUPER,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55044 entries, 0 to 55043
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   lib_mrq            55044 non-null  object 
 1   lib_mod_doss       55044 non-null  object 
 2   lib_mod            55044 non-null  object 
 3   dscom              55044 non-null  object 
 4   cnit               55044 non-null  object 
 5   tvv                55044 non-null  object 
 6   cod_cbr            55044 non-null  object 
 7   hybride            55044 non-null  object 
 8   puiss_admin_98     55044 non-null  int64  
 9   puiss_max          55044 non-null  object 
 10  typ_boite_nb_rapp  55044 non-null  object 
 11  conso_urb          55001 non-null  object 
 12  conso_exurb        55001 non-null  object 
 13  conso_mixte        55010 non-null  object 
 14  co2                55010 non-null  float64
 15  co_typ_1           54886 non-null  object 
 16  hc                 977

None

Type et pourcentage de Nan 

In [13]:
# On créé un dataframe dans lequel est stocké le type et le pourcentage de Nan de chaque variable
infos_donnees = pd.DataFrame({
    "Type": df.dtypes,
    "Pourcentage Manquant (%)": (df.isnull().sum() / len(df)) * 100
})
display(infos_donnees)

,Type,Pourcentage Manquant (%)
lib_mrq,object,0.000000
lib_mod_doss,object,0.000000
lib_mod,object,0.000000
dscom,object,0.000000
cnit,object,0.000000
tvv,object,0.000000
cod_cbr,object,0.000000
hybride,object,0.000000
puiss_admin_98,int64,0.000000
puiss_max,object,0.000000


Nous avons 22 variables de type object, nous allons regarder en combien de catégories sont divisées
ces variables.

In [18]:
# On récupère les colonnes de type Object
colonnes_object = df.select_dtypes(include='object')
# On calcule le nombre de modalité de chacune de ces variables
categories_uniques = colonnes_object.nunique()
print(categories_uniques)


lib_mrq                 46
lib_mod_doss           483
lib_mod                434
dscom                 3837
cnit                 54982
tvv                  35430
cod_cbr                 13
hybride                  2
puiss_max              232
typ_boite_nb_rapp       19
conso_urb              198
conso_exurb             89
conso_mixte            137
co_typ_1               606
hc                      74
nox                    222
hcnox                  209
ptcl                     9
champ_v9                35
date_maj                 5
Carrosserie             11
gamme                    7
dtype: int64


Avoir un aperçu de ce à quoi ressemble les catégories de certaines variables catégorielles

In [26]:
display('Marque des voitures', df['lib_mrq'].unique())
display('Gamme des voitures', df['gamme'].unique())

'Marque des voitures'

array(['ALFA-ROMEO', 'ASTON MARTIN', 'AUDI', 'BENTLEY', 'BMW', 'CADILLAC',
       'CHEVROLET', 'CITROEN', 'DACIA', 'FERRARI', 'FIAT', 'FORD',
       'HONDA', 'HYUNDAI', 'INFINITI', 'JAGUAR', 'JEEP', 'KIA', 'LADA',
       'LAMBORGHINI', 'LANCIA', 'LAND ROVER', 'LEXUS', 'LOTUS',
       'MASERATI', 'MAZDA', 'MERCEDES', 'MIA', 'MINI', 'MITSUBISHI',
       'NISSAN', 'OPEL', 'PEUGEOT', 'PORSCHE', 'RENAULT', 'ROLLS-ROYCE',
       'SEAT', 'SKODA', 'SMART', 'SSANGYONG', 'SUBARU', 'SUZUKI', 'TESLA',
       'TOYOTA', 'VOLKSWAGEN', 'VOLVO'], dtype=object)

'Gamme des voitures'

array(['MOY-SUPER', 'LUXE', 'MOY-INFER', 'INFERIEURE', 'SUPERIEURE',
       'ECONOMIQUE', 'MOY-INFERIEURE'], dtype=object)